## Experiments:
1. Sectoral Dependency
2. Financial Profile Dependency

In [6]:
import pandas as pd
import config.utils as utils
import numpy as np
import xlsxwriter as xl

eventD = utils.eventDate()
dateSetting = [(0,0),(-1,1),(-3,3),(-5,5),(0,1),(0,3),(0,5)]

In [8]:
def summary_csv():
    """
    A templete to record down results
    """
    # 23 rows and 20 columns
    summary = pd.DataFrame(index = range(23), columns = range(14))
    for i in range(2,23):
        if i%3 == 0:
            summary.loc[[i], 1] = 'S'
        elif i%3 == 1:
            summary.loc[[i], 1] = 'G'
        else:
            summary.loc[[i], 1] = 'E'
        r = (i-2)//3
        summary.loc[[i], 0] = str(dateSetting[r])

    for i in range(2,14):
        if i%3 == 0:
            summary.loc[1, [i]] = 't-value'
        elif i%3 == 1:
            summary.loc[1, [i]] = 'p-value'
        else:
            summary.loc[1, [i]] = 'coef'
        r = (i-2)//3
        summary.loc[0, [i]] = eventD['date'][r]
    return summary

---

### Industry Dependency

In [ ]:
industry_dict = {
    'Energy':50,
    'Construction_Materials':51,
    'Industrial':52,
    'Consumer_ProdServ':53,
    'Food_consumerGoods':54,
    'Financial':55,
    'Biotech':56,
    'Technology':57,
    'Utilities':59,
    'Real_Estate':60,
    'Education':63
}

In [10]:
# Sector experiments
for industry in industry_dict.keys():
    summary = summary_csv() 
    sumDummy = summary_csv() 
    interaction = summary_csv() 

    path = '/ESG/1020_journal/experiment_result/01_' + industry + '.csv'
    for i, ED in enumerate(eventD['date']):
        model = utils.EventStudy(ED)
        esgSummary, dummySummary, interSummary = model.industryES_big(industry, normalize = False, exclude = False, pillar = True)
        summary.loc[2:23, 3*i+2:3*i+4] = esgSummary.values
        sumDummy.loc[2:23, 3*i+2:3*i+4] = dummySummary.values
        interaction.loc[2:23, 3*i+2:3*i+4] = interSummary.values
    finalSummary = [summary, sumDummy, interaction]
    finalSummary = pd.concat(finalSummary)
    finalSummary.to_csv(path)

### Financial Profile Dependency

In [13]:
control_var = ['log_makvalt','BMRatio','ROA','asset_growth','HHI']
for control in control_var:
    summary = summary_csv()
    sumDummy = summary_csv()
    interaction = summary_csv()
    path = '/ESG/1020_journal/experiment_result/03_' + control + '.csv'
    for i, ED in enumerate(eventD['date']):
        model = utils.EventStudy(ED)
        esgSummary, dummySummary, interSummary = model.controlES_inter(variable = control, exclude = False, extremeGroup = True, pillar = True)
        summary.loc[2:23, 3*i+2:3*i+4] = esgSummary.values
        sumDummy.loc[2:23, 3*i+2:3*i+4] = dummySummary.values
        interaction.loc[2:23, 3*i+2:3*i+4] = interSummary.values
    finalSummary = [summary, sumDummy, interaction]
    finalSummary = pd.concat(finalSummary)
    finalSummary.to_csv(path)

---

Summarize the experiments to a excel sheet.

In [47]:

experiment_list = ['01_Energy', '01_Construction_Materials', '01_Industrial', '01_Consumer_ProdServ',
                    '01_Food_consumerGoods', '01_Financial', '01_Biotech', '01_Technology' ,
                    '01_Utilities', '01_Real_Estate', '01_Education', '02_Automobile_Parts', '02_Freight_Logistics',
                    '02_Passenger_Transportation', '02_Hotels_Entertainment', '02_Media', '02_Food_Bev_Tabacco',
                    '03_log_makvalt','03_BMRatio','03_ROA','03_asset_growth', '03_HHI', '04_hhi_dummy']

# Q3-Q1
refinitiv_Q3Q1 = {
    'E': 33,
    'S': 32,
    'G': 27
}

In [48]:
# normal or exclude
path = '/ESG/1020_journal/experiment_result/'
ESGQ3Q1 = refinitiv_Q3Q1


# write to excel summary
column = [4,7,10,13]
sector_list = []
profile_list = []
for unit in experiment_list:
    raw_file = pd.read_csv(path + unit+'.csv', index_col=0)
    unit_info = []
    for col in column:
        for row in range(20):

            triplet = [raw_file.iloc[row+2, col], raw_file.iloc[row+25, col], raw_file.iloc[row+48, col]]
            if triplet[0][0] == '*' and triplet[1][0] == '*' and triplet[2][0] == '*':
                pillar = raw_file.iloc[row+2, 1]
                info_list = [raw_file.iloc[0, col], pillar, raw_file.iloc[row+2, 0]]
                coef = float(raw_file.iloc[row+2, col-2])
                inter = float(raw_file.iloc[row+48, col-2])
                data_info = [coef, float(raw_file.iloc[row+25, col-2]), 
                            inter, ESGQ3Q1[pillar], ESGQ3Q1[pillar] * coef, 
                            (coef + inter) * ESGQ3Q1[pillar]]

                unit_info.append(info_list + data_info)

    if unit[1] != '3':
        if unit_info:
            sector_list.append([unit, 'pillar','window', 'coef','dummy','inter', 'Q3-Q1', 'CAR/subQ1', 'CAR/subQ3'])
            for i in unit_info:
                sector_list.append(i)
    else:
        if unit_info:
            profile_list.append([unit, 'pillar','window', 'coef','dummy','inter', 'Q3-Q1', 'CAR/subQ1', 'CAR/subQ3'])
            for i in unit_info:
                profile_list.append(i)
workbook = xl.Workbook('summary_hhi_industry.xlsx')
sector_sht = workbook.add_worksheet()
profile_sht = workbook.add_worksheet()

for row_num, row_data in enumerate(sector_list):
    for col_num, col_data in enumerate(row_data):
        sector_sht.write(row_num, col_num, col_data)
sector_sht.set_column('A:A', 28)
sector_sht.set_column('D:F', 15)
sector_sht.set_column('H:I', 28)


for row_num, row_data in enumerate(profile_list):
    for col_num, col_data in enumerate(row_data):
        profile_sht.write(row_num, col_num, col_data)

profile_sht.set_column('A:A', 28)
profile_sht.set_column('D:F', 15)
profile_sht.set_column('H:I', 28)


workbook.close()
